In [ ]:
from utilities.common import * 
import shutil 
from openpyxl import load_workbook
import win32com.client 
import time
def log(*s): print(*s)

st = date(2023, 8, 1)
ed = date(2024, 7, 1)
st, ed = [x + relativedelta(day=31) for x in (st, ed)]
st, ed

In [ ]:

template = fullpath("template")
src = template / "Book1.xlsx"
output = ensure_dir(fullpath("output") / "_".join([
  "rent", 
  "-".join([x.strftime('%Y%m%d') for x in (st, ed)]), 
])) 
pdfroot = ensure_dir(output / "pdf")


In [ ]:

d = st

HAKOB_CELL = "M1"
SEQNO_CELL = "M2" 
KENMEI_CELL = "C7" 
HINMEI_CELL = "C14" 
BIKO_CELL = "A32"

while d <= ed: 
  day = d + relativedelta(day=31)
  log("*"*10, "process", day, "doing...")
  
  sheetname = day.strftime('%Y年%m月分')
  hakob = day 
  seqno = (day + relativedelta(day=31)).strftime('%Y%m%d001')
  kenmei = day.strftime('%Y年%m月分家賃')
  hinmei = day.strftime('%Y年%m月分家賃')
  biko = "\n".join([
    "※賃貸物件住所：　〒863-0022 天草市栄町15-20", 
    f"※受領日：{day.strftime('%Y年%m月%d日')}", 
  ])

  filename = shutil.copy2(src, output / day.strftime("天草物件家賃領収書_%Y年%m月分.xlsx"))

  log("making...", filename.relative_to(filename.parents[1]), "doing...")
  # https://qiita.com/studio_haneya/items/7e49b616d4972fe1804d
  wb = load_workbook(filename)
  ws = wb.active
  ws.title = sheetname
  ws[HAKOB_CELL].value = hakob 
  ws[SEQNO_CELL].value = seqno 
  ws[KENMEI_CELL].value = kenmei 
  ws[HINMEI_CELL].value = hinmei
  ws[BIKO_CELL].value = biko
  wb.save(filename)
  log("making...", filename.relative_to(filename.parents[1]), "done!!")

  # https://zenn.dev/terashii/articles/5d82acb8fdda5d
  try:
    pdfname = (pdfroot / filename.name).with_suffix(".pdf")
    log("export", filename.name, "to", pdfname.name, "doing...")
    app = win32com.client.Dispatch("Excel.Application")
    # app.Visible = True 
    app.DisplayAlerts = False 
    book = app.Workbooks.Open(filename)
    time.sleep(1)
    xlTypePDF = 0
    book.ExportAsFixedFormat(xlTypePDF, str(pdfname))
    app.Quit()
    log("export", filename.name, "to", pdfname.name, "done!!")
  except Exception as e:
    log("!"*10, e)
  finally: 
    app.Quit()

  time.sleep(1)
  log("*"*10, "process", day, "done!!")
  log("")
  d += relativedelta(months=1)


In [ ]:
import shutil 
shutil.make_archive(
  output, 
  format="zip", 
  root_dir=pdfroot)